In [5]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import seaborn as sns

- o pandas é para manipulação de dados
- a numpy é para trabalharmos com algumas operações matematicas
- o dataframe serve para organizar e manipular dados tabulares, como planilhas ou tabelas de banco de dados

# Criando dataset mock

In [ ]:
import argparse
from datetime import datetime, timedelta

def sigmoid(x): return 1 / (1 + np.exp(-x))

def make_text(idx):
    adjectives = ["ótimo", "bom", "regular", "ruim", "péssimo", "excelente", "satisfatório"]
    objects = ["produto", "serviço", "entrega", "atendimento", "preço"]
    endings = ["recomendo", "não recomendo", "voltaria", "não volto", "faz o trabalho"]
    return f"{np.random.choice(adjectives)} {np.random.choice(objects)} - {np.random.choice(endings)} (id:{idx})"

def generate(n=10000, seed=42, out="data/mock_feature_engineering.csv"):
    np.random.seed(seed)
    today = datetime.today()

    # numeric
    age = np.clip((np.random.normal(40, 12, n)).astype(int), 18, 90)
    income = np.random.lognormal(mean=10.5, sigma=0.9, size=n)  # skewed
    tenure_months = np.random.exponential(scale=24, size=n).astype(int)  # in months
    num_transactions = np.random.poisson(lam=np.clip(tenure_months/3 + 1, 1, None))
    avg_transaction = np.random.gamma(shape=2.0, scale=50.0, size=n)

    # dates
    signup_days_ago = np.random.randint(0, 3650, size=n)
    signup_date = [today - timedelta(days=int(d)) for d in signup_days_ago]
    last_login_days = np.array([np.random.randint(0, max(1, d+1)) for d in signup_days_ago])
    last_login = [sd + timedelta(days=int(dl)) for sd, dl in zip(signup_date, last_login_days)]

    # categorical
    countries = ["BR", "US", "PT", "ES", "DE"]
    products = ["A", "B", "C", "D"]
    devices = ["mobile", "desktop", "tablet"]
    country = np.random.choice(countries, size=n, p=[0.5,0.2,0.15,0.1,0.05])
    product = np.random.choice(products, size=n, p=[0.4,0.3,0.2,0.1])
    device = np.random.choice(devices, size=n, p=[0.6,0.3,0.1])

    # text
    review_text = [make_text(i) for i in range(n)]

    # construct risk score and churn target (imbalanced)
    score = (
        -0.02 * age +
        -0.00001 * income +
        0.03 * (num_transactions < 2).astype(int) +
        0.04 * (tenure_months < 6).astype(int) +
        0.5 * (country == "BR").astype(int) +
        0.2 * (device == "mobile").astype(int)
    )
    churn_prob = sigmoid(-2.2 + score)  # base churn ~10%
    churn = (np.random.rand(n) < churn_prob).astype(int)

    df = pd.DataFrame({
        "customer_id": [f"CUST_{i:06d}" for i in range(n)],
        "age": age,
        "income": income,
        "tenure_months": tenure_months,
        "num_transactions": num_transactions,
        "avg_transaction": avg_transaction,
        "signup_date": signup_date,
        "last_login": last_login,
        "country": country,
        "product": product,
        "device": device,
        "review_text": review_text,
        "churn": churn
    })

    # introduce missingness
    for col, frac in [("income", 0.05), ("last_login", 0.03), ("review_text", 0.2)]:
        idx = df.sample(frac=frac, random_state=seed).index
        df.loc[idx, col] = pd.NA

    # introduce outliers
    out_idx = df.sample(frac=0.005, random_state=seed+1).index
    df.loc[out_idx, "income"] *= 10
    df.loc[out_idx, "avg_transaction"] *= 20

    # add duplicates (a few)
    duplicates = df.sample(n=max(1, n//500), random_state=seed+2)
    df = pd.concat([df, duplicates], ignore_index=True).reset_index(drop=True)

    # save
    # garante que a pasta de saída exista antes de salvar
    import os
    out_dir = os.path.dirname(out)
    if out_dir:
        os.makedirs(out_dir, exist_ok=True)
    df.to_csv(out, index=False)
    print(f"Gerado {len(df)} linhas -> {out}")



try:
    parser = argparse.ArgumentParser(description="Gerar dataset mock para feature engineering")
    parser.add_argument("--n", type=int, default=10000)
    parser.add_argument("--out", type=str, default="data/mock_feature_engineering.csv")
    parser.add_argument("--seed", type=int, default=42)
    args = parser.parse_args()
except SystemExit:
    # fallback para execução interativa (Jupyter/IPython)
    args = argparse.Namespace(n=10000, out="datasets/mock.csv", seed=42)

data = generate(n=args.n, seed=args.seed, out=args.out) # 

# adquirindo os dados

In [8]:
data = pd.read_csv("datasets/mock.csv")
data.head()

,customer_id,age,income,tenure_months,num_transactions,avg_transaction,signup_date,last_login,country,product,device,review_text,churn
0,CUST_000000,45,19719.331787,1,0,253.725397,2020-12-28 10:17:25.471900,2022-04-24 10:17:25.471900,BR,C,mobile,NaN,0
1,CUST_000001,38,27585.636050,102,46,159.304057,2016-12-11 10:17:25.471900,2021-11-12 10:17:25.471900,ES,A,desktop,satisfatório preço - voltaria (id:1),0
2,CUST_000002,47,21212.736213,38,12,8.916119,2016-03-23 10:17:25.471900,2021-08-17 10:17:25.471900,BR,A,mobile,regular entrega - não recomendo (id:2),0
3,CUST_000003,58,NaN,48,22,2.999401,2020-05-26 10:17:25.471900,2024-01-29 10:17:25.471900,PT,A,mobile,NaN,0
4,CUST_000004,37,106666.313836,28,10,42.877051,2018-07-22 10:17:25.471900,2018-09-14 10:17:25.471900,BR,B,mobile,bom produto - não recomendo (id:4),0


# Passo 2 - Valores faltantes

In [9]:
total = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys= ['Total em nulos', 'Percentual de nulos'])
missing_data.head(20)

,Total em nulos,Percentual de nulos
review_text,2006,0.20020
income,502,0.05010
last_login,302,0.03014
customer_id,0,0.00000
age,0,0.00000
num_transactions,0,0.00000
tenure_months,0,0.00000
signup_date,0,0.00000
avg_transaction,0,0.00000
country,0,0.00000



- em total, basicamente, percorre todas as colunas e conta quantos valores nulos existem em cada coluna
- em percent, calcula a porcentagem de valores nulos em cada coluna, dividindo o número de valores nulos pelo número total de entradas na coluna
- em missing_data, concatena os dois resultados anteriores em um único DataFrame, com duas colunas: 'Total' e 'Percentual', facilitando a visualização dos valores nulos em cada coluna do conjunto de dados.
- o head(20) exibe as primeiras 20 linhas do DataFrame resultante, mostrando as colunas com mais valores nulos e suas respectivas porcentagens.

In [47]:
# como o numero de valores faltantes em attributed_time é quase 81%, vamos deletar essa coluna.
data.drop(['attributed_time'], axis=1, inplace=True) # inplace=True faz a alteração diretamente no DataFrame original
data.isnull().sum().sum()
# Verificar se a coluna foi removida
print('attributed_time' in data.columns)

# Recalcular missing_data após alterações
total = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data.shape[0]).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percentual'])
print(data.isnull().sum().sum())        # total de valores faltantes
missing_data[missing_data['Total'] > 0] # colunas com valores faltantes

False
0


,Total,Percentual


In [54]:
data.is_attributed.describe()

count    2.300561e+06
mean     1.985803e-01
std      3.989313e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.000000e+00
Name: is_attributed, dtype: float64

# Normalização dos dados
- para normalizar os dados, podemos usar a técnica de Min-Max Scaling, que transforma os valores de uma coluna para um intervalo entre 0 e 1. Isso é feito subtraindo o valor mínimo da coluna e dividindo pela diferença entre o valor máximo e o valor mínimo.
- formula:
-   (valor_normalizar - valor_minimo) / (valor_maximo - valor_minimo)
- exemplo: data['coluna_normalizada'] = (data['coluna'] - data['coluna'].min()) / (data['coluna'].max() - data['coluna'].min())

# Padronização dos dados
- A padronização é igual a:
-  (valor_a_padronizar - media) / desvio_padrao
o 